In [2]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Crop Location Data
crop_location_data = pd.read_csv("data/Crop_Data_vv_vh.csv")

# Image Data
image_data = np.load('data/image_data.npy', allow_pickle=True)
image_data_df = pd.DataFrame(image_data, columns=['data'])

# Combined Data
# Check if the number of rows in both DataFrames is the same
if len(image_data_df) == len(crop_location_data):
    # Combine the DataFrames
    crop_data = pd.concat([crop_location_data, image_data_df], axis=1)
else:
    print("The number of rows in the image_data and crop_location_data files do not match. Please check the files.")

In [4]:
crop_data.head()

vh        vv Class of Land   
0  0.041293  0.114049          Rice  \
1  0.026896  0.147442          Rice   
2  0.021583  0.277267          Rice   
3  0.035070  0.154607          Rice   
4  0.041903  0.191198          Rice   

                                                data  
0  [1934, 1982, 1908, 1892, 1960, 2002, 1952, 196...  
1  [1678, 1644, 1662, 1658, 1650, 1608, 1698, 176...  
2  [1954, 2256, 2290, 2386, 2396, 2582, 2674, 260...  
3  [1871, 1802, 1750, 1687, 1759, 1728, 1927, 200...  
4  [2254, 1948, 1870, 1802, 1662, 1584, 1590, 158...

In [5]:
# Calculating the RVI
crop_data['RVI'] = (1 + np.sqrt(crop_data['vv'] / crop_data['vh'])) / (1 - np.sqrt(crop_data['vv'] / crop_data['vh']))

In [6]:
crop_data.head()

vh        vv Class of Land   
0  0.041293  0.114049          Rice  \
1  0.026896  0.147442          Rice   
2  0.021583  0.277267          Rice   
3  0.035070  0.154607          Rice   
4  0.041903  0.191198          Rice   

                                                data       RVI  
0  [1934, 1982, 1908, 1892, 1960, 2002, 1952, 196... -4.021544  
1  [1678, 1644, 1662, 1658, 1650, 1608, 1698, 176... -2.491036  
2  [1954, 2256, 2290, 2386, 2396, 2582, 2674, 260... -1.773941  
3  [1871, 1802, 1750, 1687, 1759, 1728, 1927, 200... -2.818743  
4  [2254, 1948, 1870, 1802, 1662, 1584, 1590, 158... -2.760424

In [7]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, temp_idx = next(sss.split(crop_data, crop_data['Class of Land']))
train_df = crop_data.iloc[train_idx]
temp_df = crop_data.iloc[temp_idx]

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
val_idx, test_idx = next(sss.split(temp_df, temp_df['Class of Land']))
val_df = temp_df.iloc[val_idx]
test_df = temp_df.iloc[test_idx]


print(f"Train Shape: {train_df.shape}")
print(f"Validation Shape: {val_df.shape}")
print(f"Test Shape: {test_df.shape}")

Train Shape: (420, 5)
Validation Shape: (90, 5)
Test Shape: (90, 5)


### Image Classifier

In [ ]:
def preprocess_images(df):
    # Resize images to target size (224, 224) and stack them into a single NumPy array
    resized_images = np.stack([cv2.resize(img, (224, 224)) for img in df['data'].values])
    
    # Ensure the channel dimension is included
    resized_images = np.expand_dims(resized_images, axis=-1)
    resized_images = np.repeat(resized_images, 3, axis=-1)

    # Convert categories to binary labels (assuming two categories: 'Category1' and 'Category2')
    binary_labels = df['Class of Land'].map({'Non Rice': 0, 'Rice': 1}).values

    return resized_images, binary_labels


# Preprocessing the images
train_img_data = train_df[['data', 'Class of Land']]
val_img_data = val_df[['data', 'Class of Land']]
test_img_data = test_df[['data', 'Class of Land']]

train_images, train_labels = preprocess_images(train_img_data)
val_images, val_labels = preprocess_images(val_img_data)
test_images, test_labels = preprocess_images(test_img_data)


# Create a data generator for training images
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)
val_generator = val_datagen.flow(val_images, val_labels, batch_size=32)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=32)

# Create a ResNet50 model with ImageNet weights
def efficient_model():

    model = EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights="imagenet")

    for layer in model.layers[-25:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

    # Model
    model = Model(model.input, outputs, name="EfficientNet")

    return model

# Train the model
image_classifier = efficient_model()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.003)
image_classifier.compile(
        optimizer=optimizer, 
        loss="binary_crossentropy", 
        metrics=["accuracy"])

image_classifier.fit(train_generator, 
          validation_data=val_generator, 
          epochs=15)


In [9]:
def resnet50_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Freeze the layers of the base model (ResNet50)
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers on top of the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = Dense(64, activation='relu')(x)
    x = layers.Dropout(0.2, name="top_dropout")(x)
    x = Dense(32, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)

    # Create the final model
    model = Model(inputs=base_model.input, outputs=predictions)

    return model


model = resnet50_model()

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_generator, 
          validation_data=val_generator, 
          epochs=10)

Epoch 1/10
14/14 [==============================] - 27s 2s/step - loss: 0.2935 - accuracy: 0.8786 - val_loss: 0.3776 - val_accuracy: 0.7778
Epoch 2/10
14/14 [==============================] - 23s 2s/step - loss: 0.1013 - accuracy: 0.9690 - val_loss: 0.3104 - val_accuracy: 0.8778
Epoch 3/10
14/14 [==============================] - 23s 2s/step - loss: 0.1011 - accuracy: 0.9571 - val_loss: 0.2580 - val_accuracy: 0.9333
Epoch 4/10
14/14 [==============================] - 23s 2s/step - loss: 0.0401 - accuracy: 0.9881 - val_loss: 0.2268 - val_accuracy: 0.9222
Epoch 5/10
14/14 [==============================] - 23s 2s/step - loss: 0.0380 - accuracy: 0.9952 - val_loss: 0.1866 - val_accuracy: 0.9778
Epoch 6/10
14/14 [==============================] - 22s 2s/step - loss: 0.0213 - accuracy: 0.9952 - val_loss: 0.1768 - val_accuracy: 0.9444
Epoch 7/10
14/14 [==============================] - 24s 2s/step - loss: 0.0236 - accuracy: 0.9952 - val_loss: 0.1614 - val_accuracy: 0.9444
Epoch 8/10
14/14 [==

### Random Forest Classifier

In [10]:
train_non_image_data = train_df.drop(columns=['data'])
val_non_image_data = val_df.drop(columns=['data'])
test_non_image_data = test_df.drop(columns=['data'])

X_train_non_image = train_non_image_data.drop(columns=['Class of Land']).values
y_train_non_image = train_non_image_data['Class of Land'].values

X_val_non_image = val_non_image_data.drop(columns=['Class of Land']).values
y_val_non_image = val_non_image_data['Class of Land'].values

X_test_non_image = test_non_image_data.drop(columns=['Class of Land']).values
y_test_non_image = test_non_image_data['Class of Land'].values


sc = StandardScaler()

X_train = sc.fit_transform(X_train_non_image)


In [11]:
y_train_non_image = train_non_image_data['Class of Land'].map({'Non Rice': 0, 'Rice': 1}).values
y_val_non_image = val_non_image_data['Class of Land'].map({'Non Rice': 0, 'Rice': 1}).values
y_test_non_image = test_non_image_data['Class of Land'].map({'Non Rice': 0, 'Rice': 1}).values

In [12]:
rf_clf = RandomForestClassifier(max_depth=10,
                                n_estimators=100,
                                min_samples_split=2,
                                random_state=42)

rf_clf.fit(X_train, y_train_non_image)

RandomForestClassifier(max_depth=10, random_state=42)

### Extracting features (predictions) for the training, validation, and test sets

In [13]:
train_image_features = model.predict(train_generator)
val_image_features = model.predict(val_generator)
test_image_features = model.predict(test_generator)

train_rf_features = rf_clf.predict_proba(X_train_non_image)
val_rf_features = rf_clf.predict_proba(X_val_non_image)
test_rf_features = rf_clf.predict_proba(X_test_non_image)

3/3 [==============================] - 4s 1s/step


### Combining the features

In [14]:
train_combined_features = np.hstack((train_image_features, train_rf_features))
val_combined_features = np.hstack((val_image_features, val_rf_features))
test_combined_features = np.hstack((test_image_features, test_rf_features))

In [15]:
# Combining train and validation data
X_train_combined = np.vstack((train_combined_features, val_combined_features))
y_train_combined = np.hstack((y_train_non_image, y_val_non_image))

In [16]:
# 6. Train a Logistic Regression on the combined features
from sklearn.linear_model import LogisticRegression

final_classifier = LogisticRegression(random_state=42)
final_classifier.fit(X_train_combined, y_train_combined)

# Evaluate the final classifier on the test set
test_predictions = final_classifier.predict(test_combined_features)

print("Test set performance:")
print(classification_report(test_labels, test_predictions))

Test set performance:
              precision    recall  f1-score   support

           0       0.61      0.56      0.58        45
           1       0.59      0.64      0.62        45

    accuracy                           0.60        90
   macro avg       0.60      0.60      0.60        90
weighted avg       0.60      0.60      0.60        90



In [17]:
# 6. Train a SVM on the combined features
from sklearn.svm import SVC

svm_clf = SVC(kernel='linear', C=10, gamma='auto', random_state=40)

svm_clf.fit(X_train_combined, y_train_combined)

# Evaluate the final classifier on the test set
test_predictions = svm_clf.predict(test_combined_features)

print("Test set performance:")
print(classification_report(test_labels, test_predictions))

Test set performance:
              precision    recall  f1-score   support

           0       0.58      0.69      0.63        45
           1       0.62      0.51      0.56        45

    accuracy                           0.60        90
   macro avg       0.60      0.60      0.60        90
weighted avg       0.60      0.60      0.60        90



In [18]:
# 6. Train a Random Forest on the combined features

rf_clf = RandomForestClassifier(max_depth=8,
                                n_estimators=100,
                                min_samples_split=1,
                                random_state=42)

rf_clf.fit(X_train_combined, y_train_combined)

# Evaluate the final classifier on the test set
test_predictions = rf_clf.predict(test_combined_features)
print("Test set performance:")
print(classification_report(test_labels, test_predictions))

Test set performance:
              precision    recall  f1-score   support

           0       0.57      0.58      0.57        45
           1       0.57      0.56      0.56        45

    accuracy                           0.57        90
   macro avg       0.57      0.57      0.57        90
weighted avg       0.57      0.57      0.57        90



In [19]:
# 6. Train a XGBoost Classifier on the combined features

def objective(params):
    clf = xgb.XGBClassifier(**params, use_label_encoder=False, eval_metric='error', n_jobs=-1)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = -np.mean(cross_val_score(clf, X_train_combined, y_train_combined, cv=cv, scoring='accuracy', n_jobs=-1))
    return {'loss': score, 'status': STATUS_OK}


space = {
    'n_estimators': hp.choice('n_estimators', np.arange(50, 1000, 25, dtype=int)),
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'max_depth': hp.choice('max_depth', range(1, 10)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.1, 1),
    'gamma': hp.uniform('gamma', 0, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),
}


trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials, verbose=2)\

best_params = {key: int(value) if "n_" in key or "depth" in key else value for key, value in best.items()}
xgboost_clf = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='error', n_jobs=-1)

xgboost_clf.fit(X_train_combined, y_train_combined)

y_pred = xgboost_clf.predict(test_combined_features)


# Generate a classification report
report = classification_report(test_labels, test_predictions)
print("XGBoost Classification Report:\n", report)

100%|██████████| 100/100 [00:24<00:00,  4.12trial/s, best loss: -0.6941176470588235]
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.58      0.57        45
           1       0.57      0.56      0.56        45

    accuracy                           0.57        90
   macro avg       0.57      0.57      0.57        90
weighted avg       0.57      0.57      0.57        90



### Prediction on random test samples

In [24]:
from utils import extract_image_data, get_sentinel_data

lat_long = ['10.323727047081501, 105.2516346045924', 
            '10.322364360592521, 105.27843410554115', 
            '10.321455902933202, 105.25254306225168', 
            '10.012126069934741, 105.67361318732796', 
            '10.012580298764401, 105.67361318732796']

category = ['Rice', 'Rice', 'Rice', 'Non Rice', 'Non Rice']

sample_df = pd.DataFrame({'Latitude and Longitude': lat_long, 'Class of Land': category})

In [25]:
sentinel_2_df = extract_image_data(sample_df)

Extracting data from Lat-Lon bounding box region...

...
Data Extracted!!!


In [39]:
sentinel_2_df.head()

data  category
0  [393, 398, 567, 656, 770, 747, 562, 500, 490, ...      Rice
1  [696, 718, 681, 724, 705, 600, 552, 546, 518, ...      Rice
2  [352, 349, 352, 364, 371, 340, 375, 342, 351, ...      Rice
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  Non Rice
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  Non Rice

In [36]:
sentinel_1_df = get_sentinel_data(lat_long, "2022-02-01/2022-06-30", ['vh','vv'])

In [37]:
sentinel_1_df

vh        vv Class of Land
0    0.041293  0.114049          Rice
1    0.026896  0.147442          Rice
2    0.021583  0.277267          Rice
598  0.124859  0.320639      Non Rice
599  0.054056  0.175985      Non Rice

In [38]:
# Calculating the RVI
sentinel_1_df['RVI'] = (1 + np.sqrt(sentinel_1_df['vv'] / sentinel_1_df['vh'])) / (1 - np.sqrt(sentinel_1_df['vv'] / sentinel_1_df['vh']))

In [54]:
def preprocess_images(df):
    # Resize images to target size (224, 224) and stack them into a single NumPy array
    resized_images = np.stack([cv2.resize(img, (224, 224)) for img in df['data'].values])
    
    # Ensure the channel dimension is included
    resized_images = np.expand_dims(resized_images, axis=-1)
    resized_images = np.repeat(resized_images, 3, axis=-1)

    # Convert categories to binary labels (assuming two categories: 'Category1' and 'Category2')
    binary_labels = df['category'].map({'Non Rice': 0, 'Rice': 1}).values

    return resized_images, binary_labels


img_data = sentinel_2_df[['data', 'category']]

images, labels = preprocess_images(img_data)

# Create a data generator for training images
datagen = ImageDataGenerator(rescale=1./255)

generator = datagen.flow(images, labels, batch_size=1)

In [59]:
image_features = model.predict(generator)

5/5 [==============================] - 0s 78ms/step


In [60]:
X_non_image = sentinel_1_df.drop(columns=['Class of Land']).values
y_non_image = sentinel_1_df['Class of Land'].values

non_images_features = rf_clf.predict_proba(X_non_image)

In [63]:
combined_features_ = np.hstack((image_features, non_images_features))

In [65]:
# Evaluate the final classifier on the test set
predictions = rf_clf.predict(combined_features_)

In [69]:
# Generate a classification report
y_non_image = sentinel_1_df['Class of Land'].map({'Non Rice': 0, 'Rice': 1}).values

report = classification_report(y_non_image, predictions)
print("Random Forest Classification Report:\n", report)

Random Forest Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         3

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5

